In [1]:
from collections import Counter
from itertools import permutations, combinations
from copy import deepcopy

In [2]:
class KenKen():
    def __init__(self, constraints, verbose=True):
        self._moves_made = 0
        self._board = [['.' for _ in range(9)] for _ in range(9)]
        self._constraints = [c for c in constraints if c["operator"] != "eq"]
        for contstraint in constraints:
            if contstraint["operator"] == "eq":
                for cell_row, cell_col in contstraint["cells"]:
                    self._moves_made += 1
                    self._board[cell_row][cell_col] = contstraint["total"]
        self.set_constraint_perms()
        self._possibilities = [[self.possibilities(i, j) for j in range(9)] for i in range(9)]
        self._print = verbose
        
    def print_board(self):
        if not self._print:
            return
        print("-------------------------------------------------------------------------------------------")
        for row in range(len(self._board)):
            for col in range(len(self._board[0])):
                c = " " if self._board[row][col] == "." else self._board[row][col]
                print(f"|    {c}    ", end="")
            print("|")
            for col in range(len(self._board[0])):
                if self._board[row][col] == ".":
                    c = "".join(map(str, self._possibilities[row][col]))
                    print(f"|{c:9}", end="")
                else:
                    print("|         ", end="")
            print("|\n-------------------------------------------------------------------------------------------")
            
    def exceeds_max_duplicates(self, values, max_duplicates):
        return any([occurances > max_duplicates for occurances in dict(Counter(values)).values()])
        
    def calc_max_duplicates(self, cells):
        if len(cells) == 1:
            return 1

        max_value = -1
        for cell_row, cell_col in cells:
            leftovers = [cell for cell in cells if cell[0] != cell_row and cell[1] != cell_col]
            if len(leftovers) == 0:
                num_dups = 1
            else:
                num_dups = 1 + self.calc_max_duplicates(leftovers)

            if num_dups > max_value:
                max_value = num_dups

        return max_value
    
    def dup_board(self):
        return [[self._board[row][col] for col in range(len(self._board[0]))] for row in range(len(self._board))]

    def possibilities(self, row, col):
        result = set([])
        for constraint in self._constraints:
            try:
                cell_idx = constraint["cells"].index((row, col))
            except ValueError:
                continue
            
            for possibility in constraint["perms"]:
                result.add(possibility[cell_idx])
        
        return sorted(result)
    
    def plus_permutations(self, box_num, total, max_duplicates=2):
        if box_num > total:
            return []

        if box_num == 1:
            if 1 <= total <= 9:
                return [[total]]
            else:
                return []

        results = []
        for i in range(1, 10):
            if i > (total / box_num) + 1:
                continue

            for perm in self.plus_permutations(box_num-1, total-i, max_duplicates=max_duplicates):
                new_value = sorted([i] + perm)
                if self.exceeds_max_duplicates(new_value, max_duplicates):
                    continue

                if new_value not in results:
                    results.append(new_value)
        return results
    
    def minus_permutations(self, total):
        results = []
        for i in range(1, 10):
            for j in range(1, 10):
                values = sorted([i, j])
                if i - j == total and values not in results:
                    results.append(values)
        return results
    
    def times_permutations(self, box_num, total, max_duplicates=2):
        if box_num < 1:
            return []

        if box_num == 1:
            if 1 <= total <= 9:
                return [[total]]
            else:
                return []

        results = []
        for i in range(1, 10):
            if total % i != 0:
                continue
            for perm in self.times_permutations(box_num-1, total//i, max_duplicates=max_duplicates):
                new_value = sorted([i] + perm)
                if self.exceeds_max_duplicates(new_value, max_duplicates):
                    continue

                if new_value not in results:
                    results.append(new_value)
        return results
    
    def divide_permutations(self, total):
        results = []
        for i in range(1, 10):
            for j in range(1, 10):
                if i >= j or j % i != 0 or j // i != total:
                    continue
                values = [i, j]
                if values not in results:
                    results.append(values)
        return results
    
    def set_value(self, row, col, value, reason=None):
        if self._board[row][col] != ".":
            return
        if self._print:
            print(f"Setting ({row}, {col}) to {value} because {reason}")
        self._moves_made += 1
        self._board[row][col] = value
        self.filter_perms()
                    
    def set_constraint_perms(self):
        for contstraint in self._constraints:
            if contstraint["operator"] == "add":
                max_duplicates = self.calc_max_duplicates(contstraint["cells"])
                perms = self.plus_permutations(len(contstraint["cells"]), contstraint["total"], max_duplicates)
            elif contstraint["operator"] == "sub":
                perms = self.minus_permutations(contstraint["total"])
            elif contstraint["operator"] == "mul":
                max_duplicates = self.calc_max_duplicates(contstraint["cells"])
                perms = self.times_permutations(len(contstraint["cells"]), contstraint["total"], max_duplicates)
            elif contstraint["operator"] == "div":
                perms = self.divide_permutations(contstraint["total"])
            else:
                continue
            
            contstraint["perms"] = sorted(set(p for perm in perms for p in permutations(perm)))
            
    def valid_move(self, row, col, value):
        if value not in self._possibilities[row][col]:
            return False
        
        for i in range(9):
            if i != col and self._board[row][i] == value:
                return False
            if i != row and self._board[i][col] == value:
                return False
        return True
    
    def valid_perm(self, cells, values):
        for i, value in enumerate(values):
            row, col = cells[i]
            if not self.valid_move(row, col, value):
                return False
            
        if len(set(values)) != len(values):
            for value in set(values):
                rows = set([])
                cols = set([])
                for cell_idx, (cell_row, cell_col) in enumerate(cells):
                    if values[cell_idx] != value:
                        continue
                    if cell_row in rows or cell_col in cols:
                        return False
                    
                    rows.add(cell_row)
                    cols.add(cell_col)
                    
        return True
                

    def filter_perms(self):
        for constraint in self._constraints:
            new_perms = []

            for perm in constraint["perms"]:
                if self.valid_perm(constraint["cells"], perm):
                    new_perms.append(perm)
            constraint["perms"] = new_perms
            if len(new_perms) == 0:
                raise Exception("No possible perms left")
            
            # if only one way to arrange numbers
            if len(new_perms) == 1:
                for i, (cell_row, cell_col) in enumerate(constraint["cells"]):
                    self.set_value(cell_row, cell_col, new_perms[0][i], reason="Only one perm possible")
            
            # if all arrangements have the same number for a cell
            for i in range(len(new_perms[0])):
                options_for_cell = set(perm[i] for perm in new_perms)
                if len(options_for_cell) == 1:
                    cell_row, cell_col = constraint["cells"][i]
                    self.set_value(cell_row, cell_col, list(options_for_cell)[0], reason="All perms had this")
    
    def cells_with_possibility(self, values, row_num=None, col_num=None):
        result = []
        if row_num is not None:
            for col, possibility in enumerate(self._possibilities[row_num]):
                if set(possibility).issubset(set(values)) and self._board[row_num][col] == ".":
                    result.append((row_num, col))
        else:
            for row, possibility_row in enumerate(self._possibilities):
                if set(possibility_row[col_num]).issubset(set(values)) and self._board[row][col_num] == ".":
                    result.append((row, col_num))
        return result
        
    def remove_possibilities(self, combo, except_cells=[], row_num=None, col_num=None):
        if row_num is not None:
            changes_made = False
            for i in range(9):
                if (row_num, i) in except_cells or self._board[row_num][i] != ".":
                    continue
                old_possibilities = self._possibilities[row_num][i]
                new_possibilities = [p for p in old_possibilities if p not in combo]
                self._possibilities[row_num][i] = new_possibilities
                if len(new_possibilities) == 0:
                    raise Exception(f"No possibilities left for ({row_num}, {i})")
                if len(new_possibilities) == 1:
                    self.set_value(row_num, i, new_possibilities[0], reason="Last possible value in row")
                if len(new_possibilities) == len(old_possibilities):
                    changes_made = True
            return changes_made
        else:
            changes_made = False
            for i in range(9):
                if (i, col_num) in except_cells or self._board[i][col_num] != ".":
                    continue
                old_possibilities = self._possibilities[i][col_num]
                new_possibilities = [p for p in old_possibilities if p not in combo]
                self._possibilities[i][col_num] = new_possibilities
                if len(new_possibilities) == 0:
                    raise Exception(f"No possibilities left for ({i}, {col_num})")
                if len(new_possibilities) == 1:
                    self.set_value(i, col_num, new_possibilities[0], reason="Last possible value in column")
                if len(new_possibilities) == len(old_possibilities):
                    changes_made = True
            return changes_made
    
    def filter_possibilities(self):
        for constraint in self._constraints:
            for i, (cell_row, cell_col) in enumerate(constraint["cells"]):
                all_vals = set([perm[i] for perm in constraint["perms"]])
                old_possibilities = set(self._possibilities[cell_row][cell_col])
                new_possibilities = old_possibilities.intersection(all_vals)
                self._possibilities[cell_row][cell_col] = sorted(new_possibilities)
                if len(new_possibilities) == 0:
                    raise Exception(f"No possibilities left for ({cell_row}, {cell_col}) old={old_possibilities} perm_vals={all_vals}")
                if len(new_possibilities) == 1:
                    self.set_value(cell_row, cell_col, list(new_possibilities)[0], reason="Intersection of perm and possibilities had length 1")
                
        for i in [2, 3, 4]:
            for combo in combinations(range(1, 10), i):
                combo = sorted(combo)
                for idx in range(9):
                    row_points = self.cells_with_possibility(combo, row_num=idx)
                    if len(row_points) == i:
                        self.remove_possibilities(combo, except_cells=row_points, row_num=idx)
                        
                    col_points = self.cells_with_possibility(combo, col_num=idx)
                    if len(col_points) == i:
                        self.remove_possibilities(combo, except_cells=col_points, col_num=idx)
                    
    
    def iterate(self):
        previous_value = -1
        while self._moves_made != previous_value:
            previous_value = self._moves_made
            self.filter_perms()
            self.filter_possibilities()
            
    def solved(self):
        return all([cell != "." for row in self._board for cell in row])
    
    def valid_guess(self, cells, perm):
        other_board = deepcopy(self)
        other_board._print = False
        for i, (cell_row, cell_col) in enumerate(cells):
            try:
                other_board.set_value(cell_row, cell_col, perm[i], reason="Guess")
            except:
                return False
        try:
            other_board.solve()
            return True
        except:
            return False
    
    def find_guess(self):
        for constraint in sorted(self._constraints, key=lambda x: -len(x["perms"])):
            unsolved = any([self._board[cell_row][cell_col] == "." for cell_row, cell_col in constraint["cells"]])
            if not unsolved:
                continue
            
            perms = [p for p in constraint["perms"]]
            for perm in perms:
                if self.valid_guess(constraint["cells"], perm):
                    return constraint["cells"], perm
                else:
                    constraint["perms"].remove(perm)
        raise Exception("No valid guesses")
            
    def solve(self):
        self.iterate()
        while not self.solved():
            if self._print:
                print("Need a new guess")
            cells, perm = self.find_guess()
            for i, (cell_row, cell_col) in enumerate(cells):
                self.set_value(cell_row, cell_col, perm[i], reason="Guess")
            self.iterate()

In [3]:
assert KenKen([]).exceeds_max_duplicates([1, 2, 3], 1) == False
assert KenKen([]).exceeds_max_duplicates([1, 1], 1)
assert KenKen([]).exceeds_max_duplicates([1, 1], 2) == False

assert KenKen([]).plus_permutations(3, 20, max_duplicates=1) == [[3, 8, 9], [4, 7, 9], [5, 6, 9], [5, 7, 8]]

assert KenKen([]).minus_permutations(5) == [[1, 6], [2, 7], [3, 8], [4, 9]]

assert KenKen([]).times_permutations(3, 24, max_duplicates=1) == [[1, 3, 8], [1, 4, 6], [2, 3, 4]]

assert KenKen([]).divide_permutations(4) == [[1, 4], [2, 8]]

In [4]:
constraints = [
  { "cells": [(0, 0), (0, 1)], "operator": "div", "total": 2 },
  { "cells": [(0, 2), (0, 3), (0, 4), (1, 3)], "operator": "mul", "total": 1536 },
  { "cells": [(0, 5)], "operator": "eq", "total": 7 },
  { "cells": [(0, 6), (0, 7)], "operator": "sub", "total": 2 },
  { "cells": [(0, 8), (1, 7), (1, 8), (2, 8)], "operator": "mul", "total": 432 },
  { "cells": [(1, 0), (1, 1), (2, 1)], "operator": "add", "total": 17 },
  { "cells": [(1, 2), (2, 2)], "operator": "add", "total": 8 },
  { "cells": [(1, 4), (1, 5), (2, 4)], "operator": "mul", "total": 45 },
  { "cells": [(1, 6), (2, 6), (3, 6)], "operator": "add", "total": 11 },
  { "cells": [(2, 0)], "operator": "eq", "total": 9 },
  { "cells": [(2, 3), (3, 3)], "operator": "sub", "total": 2 },
  { "cells": [(2, 5), (3, 5)], "operator": "sub", "total": 1 },
  { "cells": [(2, 7), (3, 7)], "operator": "div", "total": 2 },
  { "cells": [(3, 0), (3, 1), (3, 2)], "operator": "mul", "total": 378 },
  { "cells": [(3, 4), (4, 4)], "operator": "mul", "total": 30 },
  { "cells": [(3, 8), (4, 8)], "operator": "div", "total": 4 },
  { "cells": [(4, 0), (5, 0)], "operator": "add", "total": 6 },
  { "cells": [(4, 1), (5, 1), (5, 2)], "operator": "add", "total": 9 },
  { "cells": [(4, 2), (4, 3)], "operator": "sub", "total": 4 },
  { "cells": [(4, 5), (5, 5)], "operator": "div", "total": 2 },
  { "cells": [(4, 6)], "operator": "eq", "total": 8 },
  { "cells": [(4, 7), (5, 7), (5, 8)], "operator": "add", "total": 22 },
  { "cells": [(5, 3), (6, 2), (6, 3)], "operator": "add", "total": 13 },
  { "cells": [(5, 4), (6, 4)], "operator": "sub", "total": 1 },
  { "cells": [(5, 6), (6, 5), (6, 6)], "operator": "mul", "total": 567 },
  { "cells": [(6, 0), (6, 1)], "operator": "sub", "total": 1 },
  { "cells": [(6, 7), (6, 8)], "operator": "sub", "total": 1 },
  { "cells": [(7, 0), (7, 1)], "operator": "sub", "total": 2 },
  { "cells": [(7, 2), (7, 3), (7, 4)], "operator": "add", "total": 18 },
  { "cells": [(7, 5), (7, 6)], "operator": "sub", "total": 1 },
  { "cells": [(7, 7), (8, 5), (8, 6), (8, 7)], "operator": "mul", "total": 42 },
  { "cells": [(7, 8), (8, 8)], "operator": "sub", "total": 3 },
  { "cells": [(8, 0), (8, 1)], "operator": "sub", "total": 5 },
  { "cells": [(8, 2), (8, 3), (8, 4)], "operator": "add", "total": 18 },
]

k = KenKen(constraints)
k.solve()
k.print_board()

Setting (1, 3) to 8 because All perms had this
Setting (5, 6) to 9 because Only one perm possible
Setting (6, 5) to 9 because Only one perm possible
Setting (6, 6) to 7 because Only one perm possible
Setting (1, 4) to 9 because Last possible value in column
Setting (3, 4) to 5 because Last possible value in row
Setting (1, 5) to 5 because Only one perm possible
Setting (2, 4) to 1 because Only one perm possible
Setting (4, 4) to 6 because Only one perm possible
Setting (0, 8) to 9 because Last possible value in row
Need a new guess
Setting (5, 3) to 4 because Guess
Setting (0, 3) to 6 because All perms had this
Setting (6, 2) to 8 because Guess
Setting (0, 2) to 4 because Only one perm possible
Setting (0, 4) to 8 because Only one perm possible
Setting (6, 3) to 1 because Guess
Setting (2, 3) to 5 because Only one perm possible
Setting (3, 3) to 3 because Only one perm possible
Need a new guess
Setting (4, 1) to 2 because Guess
Setting (0, 0) to 2 because Only one perm possible
Setting